# SQLAlchemy Tutorial

In [1]:
%%capture
!pip install sqlalchemy

In [2]:
import sqlalchemy as db
import pandas as pd
db.__version__

'1.4.41'

## Getting Started

### Connecting the database

In [3]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///european_database.sqlite")

### Table object

In [4]:
conn = engine.connect()
metadata = db.MetaData()
division = db.Table('divisions', metadata, autoload=True, autoload_with=engine)

In [5]:
print(division.columns.keys())

['division', 'name', 'country']


In [6]:
print(repr(metadata.tables['divisions']))

Table('divisions', MetaData(), Column('division', TEXT(), table=<divisions>), Column('name', TEXT(), table=<divisions>), Column('country', TEXT(), table=<divisions>), schema=None)


### View the results

In [7]:
query = division.select()
exe = conn.execute(query)
result = exe.fetchmany(5)
print(result)

[('B1', 'Division 1A', 'Belgium'), ('D1', 'Bundesliga', 'Deutschland'), ('D2', '2. Bundesliga', 'Deutschland'), ('E0', 'Premier League', 'England'), ('E1', 'EFL Championship', 'England')]


### SQL query

In [8]:
print(query)

SELECT divisions.division, divisions.name, divisions.country 
FROM divisions


## Creating Tables

In [9]:
engine = db.create_engine('sqlite:///datacamp.sqlite') 
conn = engine.connect()
metadata = db.MetaData()

Student = db.Table('Student', metadata,
              db.Column('Id', db.Integer(),primary_key=True),
              db.Column('Name', db.String(255), nullable=False),
              db.Column('Major', db.String(255), default="Math"),
              db.Column('Pass', db.Boolean(), default=True)
              )

metadata.create_all(engine) 

### Inserting Data

In [10]:
query = db.insert(Student).values(Id=1, Name='Matthew', Major="English", Pass=True) 
Result = conn.execute(query)


In [11]:
output = conn.execute(db.select([Student])).fetchall()
print(output)

[(1, 'Matthew', 'English', True)]


### Insert Many

In [12]:
query = db.insert(Student) 
values_list = [{'Id':'2', 'Name':'Nisha', 'Major':"Science", 'Pass':False},
               {'Id':'3', 'Name':'Natasha', 'Major':"Math", 'Pass':True},
               {'Id':'4', 'Name':'Ben', 'Major':"English", 'Pass':False}]
Result = conn.execute(query,values_list)

In [13]:
output = conn.execute(db.select([Student])).fetchall()
print(output)

[(1, 'Matthew', 'English', True), (2, 'Nisha', 'Science', False), (3, 'Natasha', 'Math', True), (4, 'Ben', 'English', False)]


### Simple SQL Query

In [14]:
output = conn.execute("SELECT * FROM Student")
print(output.fetchall())

[(1, 'Matthew', 'English', 1), (2, 'Nisha', 'Science', 0), (3, 'Natasha', 'Math', 1), (4, 'Ben', 'English', 0)]


In [15]:
output = conn.execute("SELECT Name, Major FROM Student WHERE Pass = True")
print(output.fetchall())

[('Matthew', 'English'), ('Natasha', 'Math')]


## Using SQLAlchemy API

### WHERE

In [16]:
query = Student.select().where(Student.columns.Major == 'English')
output = conn.execute(query)
print(output.fetchall())

[(1, 'Matthew', 'English', True), (4, 'Ben', 'English', False)]


### AND

In [17]:
query = Student.select().where(db.and_(Student.columns.Major == 'English', Student.columns.Pass != True))
output = conn.execute(query)
print(output.fetchall())

[(4, 'Ben', 'English', False)]


### ORDER BY

In [18]:
query = Student.select().order_by(db.desc(Student.columns.Name))
output = conn.execute(query)
print(output.fetchall())

[(2, 'Nisha', 'Science', False), (3, 'Natasha', 'Math', True), (1, 'Matthew', 'English', True), (4, 'Ben', 'English', False)]


### IN

In [19]:
query = Student.select().where(Student.columns.Major.in_(['English','Math']))
output = conn.execute(query)
print(output.fetchall())

[(1, 'Matthew', 'English', True), (3, 'Natasha', 'Math', True), (4, 'Ben', 'English', False)]


### LIMIT

In [20]:
query = Student.select().limit(3)
output = conn.execute(query)
print(output.fetchall())

[(1, 'Matthew', 'English', True), (2, 'Nisha', 'Science', False), (3, 'Natasha', 'Math', True)]


#### SUM

In [21]:

query = db.select([db.func.sum(Student.columns.Id)])
output = conn.execute(query)
print(output.fetchall())

[(10,)]


### GROUP BY

In [22]:

query = db.select([db.func.sum(Student.columns.Id),Student.columns.Major]).group_by(Student.columns.Pass)
output = conn.execute(query)
print(output.fetchall())

[(6, 'Science'), (4, 'English')]


### DISTINCT

In [23]:

query = db.select([Student.columns.Major.distinct()])
output = conn.execute(query)
print(output.fetchall())

[('English',), ('Science',), ('Math',)]


## Output to Pandas DataFrame

In [24]:
query = Student.select().where(Student.columns.Major.in_(['English','Math']))
output = conn.execute(query)
results = output.fetchall()


In [28]:
data = pd.DataFrame(results)
data.columns = results[0].keys()
data

,Id,Name,Major,Pass
0,1,Matthew,English,True
1,3,Natasha,Math,True
2,4,Ben,English,False


## Data Analytics With SQLAlchemy

### Setting up Two tables


In [36]:
engine = create_engine("sqlite:///european_database.sqlite")
conn = engine.connect()
metadata = db.MetaData()
division = db.Table('divisions', metadata, autoload=True, autoload_with=engine)
match = db.Table('matchs', metadata, autoload=True, autoload_with=engine)

### Query

In [37]:
query = db.select([division,match]).\
		select_from(division.join(match,division.columns.division == match.columns.Div)).\
		where(db.and_(division.columns.division == "E1", match.columns.season == 2009 )).\
		order_by(match.columns.HomeTeam)
output = conn.execute(query)
results = output.fetchall()

### To DataFrame

In [38]:
data = pd.DataFrame(results)
data.columns = results[0].keys()
data

### Data Visualization

In [39]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")


In [40]:
f, ax = plt.subplots(figsize=(15, 6))
plt.xticks(rotation=90)
sns.set_color_codes("pastel")
sns.barplot(x="HomeTeam", y="FTHG", data=data,
            label="Home Team Goals", color="b")

sns.barplot(x="HomeTeam", y="FTAG", data=data,
            label="Away Team Goals", color="r")
ax.legend(ncol=2, loc="upper left", frameon=True)
ax.set(ylabel="", xlabel="")
sns.despine(left=True, bottom=True)

## SQL TO CSV

In [41]:
output = conn.execute("SELECT * FROM matchs WHERE HomeTeam LIKE 'Norwich'")
results = output.fetchall()

In [42]:
data = pd.DataFrame(results)
data.columns = results[0].keys()
data.to_csv("SQl_result.csv",index=False)

## CSV To SQL Table

In [43]:
engine = create_engine("sqlite:///datacamp.sqlite")

In [44]:
df = pd.read_csv('Stock Exchange Data.csv')
df.to_sql(con=engine, name="Stock_price", if_exists='replace',index=False)

In [45]:
conn = engine.connect()
metadata = db.MetaData()
stock = db.Table('Stock_price', metadata, autoload=True, autoload_with=engine)

In [46]:
print(stock.columns.keys())

In [47]:
query = stock.select()
exe = conn.execute(query)
result = exe.fetchmany(5)
for r in result:
    print(r)

## SQL Table Management

### Updating the values in table

In [48]:
Student = db.Table('Student', metadata, autoload=True, autoload_with=engine)
query = Student.update().values(Pass = True).where(Student.columns.Name == "Nisha")
results = conn.execute(query)

In [49]:
output = conn.execute(Student.select()).fetchall()
data = pd.DataFrame(output)
data.columns = output[0].keys()
data

### Delete Values

In [ ]:
Student = db.Table('Student', metadata, autoload=True, autoload_with=engine)
query = Student.delete().where(Student.columns.Name == "Ben")
results = conn.execute(query)

In [51]:
output = conn.execute(Student.select()).fetchall()
data = pd.DataFrame(output)
data.columns = output[0].keys()
data

### Delete tables

#### Close all the executions

In [58]:
results.close()
exe.close()

#### Droping the tables

In [59]:
Student.drop(engine) #drop single table

In [60]:
metadata.drop_all(engine) #drop all tables

## Thank you